In [ ]:
import groups

import numpy as np
import pandas as pd
import os.path as op
import re

import matplotlib.pyplot as plt
import seaborn as sns

import importlib

In [ ]:
importlib.reload(groups)

In [ ]:
fn = op.join("data", "Balance of skills CSS 2024(1-83).xlsx")

assert op.exists(fn)

In [ ]:
## here, define all column names (take them from opening the Excel and copy/paste)

col_name = 'Name2'
col_email = 'E-mail address'

cols_skills = ['Programming skills2', 'Domain-specific knowledge/skills']

# should also be read from the file and stored in the dataframe, to be used later or at least included in the output of the group assignment
cols_extra = ['What is your (BSc) background? (E.g., informatics, psychobiology, ...)', 
              'Do you know what topic you\'d like to work in, if you had the choice? (E.g., climate change, financial markets, molecular biology, ...) Mention as many keywords as possible, such as multiple topics...']

col_interests = cols_extra[-1]  # this column may be used in the optimization process (textual similarity)

In [ ]:
df = groups.read_student_records(fn, cols_skills, col_name=col_name, col_email=col_email,
                                 col_background=cols_extra[0], 
                                 col_interests=cols_extra[1], 
                                 to_replace=[groups._default_skill_replacement, groups._default_application_domain_expertise_replacement], verbose=1)

## Find potential duplicate names

In [ ]:
from difflib import SequenceMatcher

def text_similarity(a: str, b: str) -> float:
    """Return a number between 0 and 1 indicating how similar the two given strings are."""
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
name_similarity_threshold = 0.8  # the higher this number, the more conservative is the filtering of duplicate names

In [ ]:
name_sims = []  # list of (name_index1, name_index2, name1, name2, similarity-score)

names = df[col_name].to_list()

for nix1, row1 in df[[col_name]].iterrows():
    name1 = row1[col_name]
    for nix2, row2 in df[[col_name]].iterrows():
        name2 = row2[col_name]
        if nix2 > nix1:  # prevent checking a name pair in both directions
            sim = text_similarity(name1, name2)

            name_sims.append((nix1, nix2, name1, name2, sim))

name_sims = list(sorted(name_sims, key=lambda tup: tup[-1], reverse=True))

# all name pairs that are very similar (also some pairs below the `name_similarity_threshold` so one can eyeball what a good threshold would actually be):
[n for n in name_sims if n[-1] > 0.8 * name_similarity_threshold]

In [ ]:
## actually REMOVE the duplicate names (always remove the earlier occurrence)

for nix1, nix2, name1, name2, sim in name_sims:
    if sim > name_similarity_threshold:
        if nix1 in df.index:
            df.drop(nix1, axis='index', inplace=True)
            print(f'Removed row with index={nix1}, which had {col_name}={name1}, because it was too similar to index={nix2}, {col_name}={name2} (similarity: {sim})')
        else:
            print(f'Did not remove row with index={nix1} because it no longer existed in the dataframe')

## Make groups

In [ ]:
print(f'Will make groups with {len(df)} people.')

In [ ]:
# show a heatmap of the programming skills versys domain expertise
sns.heatmap(pd.crosstab(df[cols_skills[0]], df[cols_skills[1]]))
plt.show()

In [ ]:
_target_group_size = 4

In [ ]:
group_sizes = groups.infer_group_size_array(len(df), _target_group_size)

print(len(group_sizes))

group_sizes

In [ ]:
# run an optimization of group compositions based on balancing the skills as well as aligning the interests 
# (may take 1-15 minutes; e.g. for 82 students it took around 10 minutes for `max_iter=2500`)
fitness, group_assignment = groups.optimize_assignment_for_diversity(df, cols_skills, _target_group_size, steepness=(180., 30.), max_iter=3500, max_contig_no_improvements=50, interests_weight=0.5, 
                                                                     col_interests=col_interests, verbose=1)

In [ ]:
# convergence plot of the optimization procedure; here you'd want to see that the fitness has sufficiently converged to its maximum value and that no further appreciable
# growth can be expected. Otherwise, increase `max_iter` in the above optimization. 
plt.plot(range(len(groups._fitness_over_epochs)), groups._fitness_over_epochs, '-ok')
plt.xlabel('Epoch')
plt.ylabel('Fitness')
plt.show()

In [ ]:
group_assignment  # solution of assignment of student indices (i.e., row numbers in the dataframe) organized into groups

In [ ]:
output_dir = 'results'

In [ ]:
results_label = op.basename(fn)[:op.basename(fn).index('.')].replace(' ', '_').lower()

results_label

In [ ]:
### This prints all groups in a simple text (also to file), but this is only insightful for small numbers of students.
### It is more meant for completeness, can be removed when this output is made nicer such as in a dataframe/xlsx file.

studrowix_to_groupix = dict()

for gix, stud_ixs in enumerate(group_assignment):
    with open(op.join(output_dir, f'group{gix}_{results_label}.txt'), 'w') as fout:
        print(f'\tGroup {gix}:')

        for six in stud_ixs:
            studrowix_to_groupix[six] = gix

            print(f' - Student (row {six} in data file):\n{df.loc[six, [col_name, col_email] + cols_extra].to_dict()}')
            fout.write(f' - Student (row {six} in data file):\n{df.loc[six, [col_name, col_email] + cols_extra].to_dict()}'.encode("ascii", "ignore").decode())

In [ ]:
df_out = df.copy(deep=True)

df_out['Group ID'] = [studrowix_to_groupix[six] for six in df_out.index]

df_out

In [ ]:
fn_out_xlsx = op.join(output_dir, f'group_assignment_{results_label}_minimal.xlsx')

df_out.to_excel(op.join(output_dir, f'group_assignment_{results_label}.xlsx'))
df_out[[col_name, col_email, 'Group ID']].to_excel(fn_out_xlsx)

print(f'Results written to {fn_out_xlsx}.')

## Tweaking

Here you can find some names or group compositions and see if you can co-locate some names, or find groups which are not diverse enough, etc.

In [ ]:
def get_rows_which_contain(df, col_name: str, col_val: str):
    return df.loc[list(map(lambda name: col_val in name, df[col_name]))]

def get_rows_where_equals(df, col_name: str, col_val):
    return df.loc[df[col_name] == col_val]

In [ ]:
# get_rows_which_contain(df_out, 'Name', 'Elizabeth')  # example

### Compare two groups

In [ ]:
get_rows_where_equals(df_out, 'Group ID', 6)